In [1]:
# Импорт необходимых библиотек
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

---
## 1. Загрузка данных и их предварительный осмотр
---

In [2]:
# Путь к файлам
kirovskiy_rayon_csv = 'Kirov.csv'
ns_rayon_csv = 'NC.csv'

In [3]:
# Явно указываем разделитель и загружаем данные, игнорируя первую строку с некорректными заголовками
kirovskiy_rayon_df = pd.read_csv(kirovskiy_rayon_csv, delimiter=';', skiprows=1)
ns_rayon_df = pd.read_csv(ns_rayon_csv, delimiter=';', skiprows=1)

# Переименовываем столбцы вручную (для Кировского и Н-С районов)
kirovskiy_rayon_columns = ['Год', 'Месяц', 'Цена', 'Ключевая ставка ЦБ', 'Акции застройщиков', 
                           'Средневзвеш ставка по ипотеке', 'Перегрев', 'Курс доллара', 
                           'Инфляция', 'Инвесторы', 'Ставки по депозитам']

ns_rayon_columns = ['Год', 'Месяц', 'Цена', 'Ключевая ставка ЦБ', 'Акции застройщиков', 
                    'Пандемия', 'Курс доллара', 'Инфляция', 'Уровень дохода', 
                    'Дефицит раб силы', 'Инвесторы', 'Ставки по депозитам', 'Спекуляции']

# Присваиваем новые заголовки
kirovskiy_rayon_df.columns = kirovskiy_rayon_columns
ns_rayon_df.columns = ns_rayon_columns

# Удаляем первую строку, так как она содержит дублирующие заголовки
kirovskiy_rayon_df = kirovskiy_rayon_df.iloc[1:]
ns_rayon_df = ns_rayon_df.iloc[1:]


---
## 2. Преобразование строковых значений в числовой формат
---
Преобразуем строковые столбцы в числовой формат

In [4]:
# Столбцы, которые нужно преобразовать в числовой формат для Кировского района
columns_to_fix_kirovskiy = ['Ключевая ставка ЦБ', 'Акции застройщиков', 'Средневзвеш ставка по ипотеке', 
                            'Перегрев', 'Курс доллара', 'Инфляция', 'Инвесторы', 'Ставки по депозитам']

In [5]:
# Столбцы, которые нужно преобразовать в числовой формат для Н-С района
columns_to_fix_ns = ['Ключевая ставка ЦБ', 'Акции застройщиков', 'Пандемия', 'Курс доллара', 
                     'Инфляция', 'Уровень дохода', 'Дефицит раб силы', 'Инвесторы', 
                     'Ставки по депозитам', 'Спекуляции']

In [6]:
# Заменяем запятые на точки и преобразуем в float для Кировского района
for col in columns_to_fix_kirovskiy:
    # Преобразуем столбец в строковый формат, если это необходимо
    if kirovskiy_rayon_df[col].dtype != 'object':
        kirovskiy_rayon_df[col] = kirovskiy_rayon_df[col].astype(str)
    # Заменяем запятые на точки и преобразуем в float
    kirovskiy_rayon_df[col] = kirovskiy_rayon_df[col].str.replace(',', '.').astype(float)

# Аналогично для Н-С района
for col in columns_to_fix_ns:
    if ns_rayon_df[col].dtype != 'object':
        ns_rayon_df[col] = ns_rayon_df[col].astype(str)
    ns_rayon_df[col] = ns_rayon_df[col].str.replace(',', '.').astype(float)


In [7]:
# Заменяем запятые на точки и преобразуем столбец "Цена" в числовой формат
kirovskiy_rayon_df['Цена'] = kirovskiy_rayon_df['Цена'].str.replace(',', '.').astype(float)
ns_rayon_df['Цена'] = ns_rayon_df['Цена'].str.replace(',', '.').astype(float)


In [8]:
# Проверим типы данных
print("Типы данных для Кировского района:")
print(kirovskiy_rayon_df.dtypes)
print("Типы данных для Н-С района:")
print(ns_rayon_df.dtypes)

Типы данных для Кировского района:
Год                                int64
Месяц                             object
Цена                             float64
Ключевая ставка ЦБ               float64
Акции застройщиков               float64
Средневзвеш ставка по ипотеке    float64
Перегрев                         float64
Курс доллара                     float64
Инфляция                         float64
Инвесторы                        float64
Ставки по депозитам              float64
dtype: object
Типы данных для Н-С района:
Год                      int64
Месяц                   object
Цена                   float64
Ключевая ставка ЦБ     float64
Акции застройщиков     float64
Пандемия               float64
Курс доллара           float64
Инфляция               float64
Уровень дохода         float64
Дефицит раб силы       float64
Инвесторы              float64
Ставки по депозитам    float64
Спекуляции             float64
dtype: object


---
## 3. Обучение моделей дерева решений для Кировского и Н-С районов
---
Обучение модели для Кировского района

In [9]:
train_kirovskiy = kirovskiy_rayon_df.dropna(subset=['Цена'])
X_train_kirovskiy = train_kirovskiy.drop(['Цена', 'Год', 'Месяц'], axis=1)
y_train_kirovskiy = train_kirovskiy['Цена']

In [10]:
tree_model_kirovskiy = DecisionTreeRegressor(random_state=42)
tree_model_kirovskiy.fit(X_train_kirovskiy, y_train_kirovskiy)

DecisionTreeRegressor(random_state=42)

In [11]:
# Обучение модели для Н-С района
train_ns = ns_rayon_df.dropna(subset=['Цена'])
X_train_ns = train_ns.drop(['Цена', 'Год', 'Месяц'], axis=1)
y_train_ns = train_ns['Цена']

In [12]:
tree_model_ns = DecisionTreeRegressor(random_state=42)
tree_model_ns.fit(X_train_ns, y_train_ns)

DecisionTreeRegressor(random_state=42)

---
## 4. Предсказание пропущенных значений и обновление данных
---
Предсказание пропущенных значений для Кировского района

In [13]:
test_kirovskiy = kirovskiy_rayon_df[kirovskiy_rayon_df['Цена'].isna()].drop(['Цена', 'Год', 'Месяц'], axis=1)
predicted_kirovskiy = tree_model_kirovskiy.predict(test_kirovskiy)
kirovskiy_rayon_df.loc[kirovskiy_rayon_df['Цена'].isna(), 'Цена'] = predicted_kirovskiy

In [14]:
# Предсказание пропущенных значений для Н-С района
test_ns = ns_rayon_df[ns_rayon_df['Цена'].isna()].drop(['Цена', 'Год', 'Месяц'], axis=1)
predicted_ns = tree_model_ns.predict(test_ns)
ns_rayon_df.loc[ns_rayon_df['Цена'].isna(), 'Цена'] = predicted_ns

---
## 5. Вывод обновленных данных
---
Выводим обновленные данные

In [15]:
print("Обновленные данные для Кировского района:")
print(kirovskiy_rayon_df.head())
print("Обновленные данные для Н-С района:")
print(ns_rayon_df.head())

Обновленные данные для Кировского района:
    Год    Месяц     Цена  Ключевая ставка ЦБ  Акции застройщиков  \
1  2019   январь  61988.0                7.75                 0.0   
2  2019  февраль  64618.0                7.75                 0.0   
3  2019     март  64763.0                7.75                 0.0   
4  2019   апрель  65985.0                7.75                 0.0   
5  2019      май  64944.0                7.75                 0.0   

   Средневзвеш ставка по ипотеке  Перегрев  Курс доллара  Инфляция  Инвесторы  \
1                           9.75       0.0         66.51      1.01        0.0   
2                           9.99       0.0         65.81      0.44        0.0   
3                          10.26       0.0         65.09      0.32        0.0   
4                          10.58       0.0         64.60      0.29        0.0   
5                          10.76       0.0         64.81      0.34        0.0   

   Ставки по депозитам  
1                 7.54  
2     